In [ ]:
from torch import nn, save, load
import torch
from PIL import Image

In [ ]:
from torch.optim import Adam

In [ ]:
from torch.utils.data import DataLoader #load dataset from pytorch

In [ ]:
from torchvision import datasets  

In [ ]:
from torchvision.transforms import ToTensor #convert image to tensor

In [ ]:
#get data

train = datasets.MNIST(root="data", download=True, train=True, transform=ToTensor()) #10 classes-> 0-9
train_loader=DataLoader(train,32) #32-> batches of 32
test = datasets.MNIST(root="data", download=True, train=False, transform=ToTensor()) #10 classes-> 0-9
test_loader=DataLoader(train,32) #32-> batches of 32

In [ ]:
class ImageClassifier(nn.Module):
  def __init__(self):
    super().__init__() #subclass this model
    self.model= nn.Sequential(  #all layers are defined here
        nn.Conv2d(1,32,(3,3)), #1-> black nd white images, 32-> batch, 3*3-> size
        nn.ReLU(),
        nn.Conv2d(32,64,(3,3)),
        nn.ReLU(),
        nn.Conv2d(64,64,(3,3)),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(64*(28-6)*(28-6), 10)     # (28-6)-> in each layer we'll be shaping 2 pixel from width and height-> 6---> final size-> 1*28*28-> 10 O/P-> 10 classes
    
    )

  def forward(self,x):
    return self.model(x)

#INSTANCE OF NN, LOSS, OPTIMIZER
clf = ImageClassifier().to('cuda') #extending to GPU
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn= nn.CrossEntropyLoss()

#Training Flow
if __name__ == "__main__":
  for epoch in range(10):
    for batch in train_loader:
      X,y= batch
      X,y=X.to('cuda'), y.to('cuda')
      yhat=clf(X) #prediction
      loss= loss_fn(yhat,y)

      #apply backpropagation
      opt.zero_grad()
      loss.backward()
      opt.step() #gradient descent

    print(f"Epoch:{epoch} loss is {loss.item()}")
    with open ('model_state.pt','wb') as f:
      save(clf.state_dict(),f)

Epoch:0 loss is 0.11470768600702286
Epoch:1 loss is 0.0026096345391124487
Epoch:2 loss is 0.0019300616113469005
Epoch:3 loss is 6.27452100161463e-05
Epoch:4 loss is 2.822665192070417e-05
Epoch:5 loss is 8.82285603438504e-05
Epoch:6 loss is 4.100267688045278e-05
Epoch:7 loss is 4.518685727816774e-06
Epoch:8 loss is 0.000560738320928067
Epoch:9 loss is 2.1606624045489298e-07


In [ ]:
class ImageClassifier_nohid(nn.Module):
  def __init__(self):
    super().__init__() #subclass this model
    self.model= nn.Sequential(  #all layers are defined here
        nn.Flatten(),
        nn.Linear(28*28, 10)     # (28-6)-> in each layer we'll be shaping 2 pixel from width and height-> 6---> final size-> 1*28*28-> 10 O/P-> 10 classes
    
    )

  def forward(self,x):
    return self.model(x)

#INSTANCE OF NN, LOSS, OPTIMIZER
clf = ImageClassifier().to('cuda') #extending to GPU
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn= nn.CrossEntropyLoss()

#Training Flow
if __name__ == "__main__":
  for epoch in range(10):
    train_accuracy=0
    for batch in train_loader:
      X,y= batch
      X,y=X.to('cuda'), y.to('cuda')
      yhat=clf(X) #prediction
      loss= loss_fn(yhat,y)
      yhat = yhat.data.max(dim=1, keepdim=True)[1]
      train_accuracy += yhat.eq(y.data.view_as(yhat)).cpu().sum()

      #apply backpropagation
      opt.zero_grad()
      loss.backward()
      opt.step() #gradient descent

    train_accuracy = train_accuracy/len(train_loader.dataset)*100
    print(f"Epoch:{epoch} loss is {loss.item()}, train accuracy is {train_accuracy}")
    with open ('model_state.pt','wb') as f:
      save(clf.state_dict(),f)

Epoch:0 loss is 0.031427960842847824, train accuracy is 96.19499969482422
Epoch:1 loss is 0.0009594273869879544, train accuracy is 98.63500213623047
Epoch:2 loss is 0.0009347838349640369, train accuracy is 99.17500305175781
Epoch:3 loss is 0.0016275722300633788, train accuracy is 99.42832946777344
Epoch:4 loss is 7.979183465067763e-06, train accuracy is 99.5816650390625
Epoch:5 loss is 3.9447626477340236e-05, train accuracy is 99.67832946777344
Epoch:6 loss is 7.092548003129195e-06, train accuracy is 99.72166442871094
Epoch:7 loss is 4.928247108182404e-06, train accuracy is 99.76499938964844
Epoch:8 loss is 1.6726382909837412e-06, train accuracy is 99.78500366210938
Epoch:9 loss is 5.364385629036406e-07, train accuracy is 99.79833221435547


In [ ]:
class ImageClassifier_onehid(nn.Module):
  def __init__(self):
    super().__init__() #subclass this model
    self.model= nn.Sequential(  #all layers are defined here
        nn.Flatten(),
        nn.Linear(28*28, 500),     # (28-6)-> in each layer we'll be shaping 2 pixel from width and height-> 6---> final size-> 1*28*28-> 10 O/P-> 10 classes
        nn.Linear(500,10)
    
    )

  def forward(self,x):
    return self.model(x)

#INSTANCE OF NN, LOSS, OPTIMIZER
clf = ImageClassifier().to('cuda') #extending to GPU
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn= nn.CrossEntropyLoss()

#Training Flow
if __name__ == "__main__":
  for epoch in range(10):
    train_accuracy=0
    for batch in train_loader:
      X,y= batch
      X,y=X.to('cuda'), y.to('cuda')
      yhat=clf(X) #prediction
      loss= loss_fn(yhat,y)
      yhat = yhat.data.max(dim=1, keepdim=True)[1]
      train_accuracy += yhat.eq(y.data.view_as(yhat)).cpu().sum()

      #apply backpropagation
      opt.zero_grad()
      loss.backward()
      opt.step() #gradient descent

    train_accuracy = train_accuracy/len(train_loader.dataset)*100
    print(f"Epoch:{epoch} loss is {loss.item()}, train accuracy is {train_accuracy}")
    with open ('model_state.pt','wb') as f:
      save(clf.state_dict(),f)

Epoch:0 loss is 0.0222284235060215, train accuracy is 95.85000610351562
Epoch:1 loss is 0.0014848064165562391, train accuracy is 98.55833435058594
Epoch:2 loss is 0.0005562488222494721, train accuracy is 99.1550064086914
Epoch:3 loss is 7.533289317507297e-05, train accuracy is 99.38666534423828
Epoch:4 loss is 2.938209581770934e-05, train accuracy is 99.55000305175781
Epoch:5 loss is 4.899513805867173e-05, train accuracy is 99.64500427246094
Epoch:6 loss is 2.348536509089172e-05, train accuracy is 99.72833251953125
Epoch:7 loss is 0.00032784478389658034, train accuracy is 99.77833557128906
Epoch:8 loss is 6.119492172729224e-05, train accuracy is 99.81333923339844
Epoch:9 loss is 3.532365371938795e-05, train accuracy is 99.81666564941406


In [ ]:
class ImageClassifier_twohid(nn.Module):
  def __init__(self):
    super().__init__() #subclass this model
    self.model= nn.Sequential(  #all layers are defined here
        nn.Flatten(),
        nn.Linear(28*28, 200),     # (28-6)-> in each layer we'll be shaping 2 pixel from width and height-> 6---> final size-> 1*28*28-> 10 O/P-> 10 classes
        nn.Linear(200,300),
        nn.Linear(300,10)
    )

  def forward(self,x):
    return self.model(x)

#INSTANCE OF NN, LOSS, OPTIMIZER
clf = ImageClassifier().to('cuda') #extending to GPU
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn= nn.CrossEntropyLoss()

##Training Flow
if __name__ == "__main__":
  for epoch in range(10):
    train_accuracy=0
    for batch in train_loader:
      X,y= batch
      X,y=X.to('cuda'), y.to('cuda')
      yhat=clf(X) #prediction
      loss= loss_fn(yhat,y)
      yhat = yhat.data.max(dim=1, keepdim=True)[1]
      train_accuracy += yhat.eq(y.data.view_as(yhat)).cpu().sum()

      #apply backpropagation
      opt.zero_grad()
      loss.backward()
      opt.step() #gradient descent

    train_accuracy = train_accuracy/len(train_loader.dataset)*100
    print(f"Epoch:{epoch} loss is {loss.item()}, train accuracy is {train_accuracy}")
    with open ('model_state.pt','wb') as f:
      save(clf.state_dict(),f)

Epoch:0 loss is 0.047737542539834976, train accuracy is 96.20500183105469
Epoch:1 loss is 0.0020163198933005333, train accuracy is 98.63999938964844
Epoch:2 loss is 0.00024294460308738053, train accuracy is 99.1933364868164
Epoch:3 loss is 0.00012687566049862653, train accuracy is 99.375
Epoch:4 loss is 0.00039435282815247774, train accuracy is 99.55333709716797
Epoch:5 loss is 9.561596925777849e-06, train accuracy is 99.67832946777344
Epoch:6 loss is 0.00037592771695926785, train accuracy is 99.6883316040039
Epoch:7 loss is 1.13874575617956e-05, train accuracy is 99.788330078125
Epoch:8 loss is 3.9487906633439707e-07, train accuracy is 99.77166748046875
Epoch:9 loss is 3.661833943624515e-06, train accuracy is 99.8066635131836


In [ ]:
class ImageClassifier_threehid(nn.Module):
  def __init__(self):
    super().__init__() #subclass this model
    self.model= nn.Sequential(  #all layers are defined here
        nn.Flatten(),
        nn.Linear(28*28, 125),    # (28-6)-> in each layer we'll be shaping 2 pixel from width and height-> 6---> final size-> 1*28*28-> 10 O/P-> 10 classes
        nn.Linear(125,250),
        nn.Linear(250,125),
        nn.Linear(125,10)
    )

  def forward(self,x):
    return self.model(x)

#INSTANCE OF NN, LOSS, OPTIMIZER
clf = ImageClassifier().to('cuda') #extending to GPU
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn= nn.CrossEntropyLoss()

#Training Flow
if __name__ == "__main__":
  for epoch in range(10):
    train_accuracy=0
    for batch in train_loader:
      X,y= batch
      X,y=X.to('cuda'), y.to('cuda')
      yhat=clf(X) #prediction
      loss= loss_fn(yhat,y)
      yhat = yhat.data.max(dim=1, keepdim=True)[1]
      train_accuracy += yhat.eq(y.data.view_as(yhat)).cpu().sum()

      #apply backpropagation
      opt.zero_grad()
      loss.backward()
      opt.step() #gradient descent

    train_accuracy = train_accuracy/len(train_loader.dataset)*100
    print(f"Epoch:{epoch} loss is {loss.item()}, train accuracy is {train_accuracy}")
    with open ('model_state.pt','wb') as f:
      save(clf.state_dict(),f)

Epoch:0 loss is 0.01179519947618246, train accuracy is 96.2066650390625
Epoch:1 loss is 0.0044312491081655025, train accuracy is 98.67333221435547
Epoch:2 loss is 0.0012728354195132852, train accuracy is 99.22833251953125
Epoch:3 loss is 0.0006013456732034683, train accuracy is 99.4433364868164
Epoch:4 loss is 0.0012107209768146276, train accuracy is 99.57167053222656
Epoch:5 loss is 1.608794991625473e-05, train accuracy is 99.6683349609375
Epoch:6 loss is 0.00020626203331630677, train accuracy is 99.74333190917969
Epoch:7 loss is 1.2336973668425344e-05, train accuracy is 99.7550048828125
Epoch:8 loss is 4.105144398636185e-06, train accuracy is 99.8066635131836
Epoch:9 loss is 0.043505843728780746, train accuracy is 99.76499938964844


In [ ]:
class ImageClassifier_fourhid(nn.Module):
  def __init__(self):
    super().__init__() #subclass this model
    self.model= nn.Sequential(  #all layers are defined here
        nn.Flatten(),
        nn.Linear(28*28, 100),    # (28-6)-> in each layer we'll be shaping 2 pixel from width and height-> 6---> final size-> 1*28*28-> 10 O/P-> 10 classes
        nn.Linear(100,100),
        nn.Linear(100,100),
        nn.Linear(100,200),
        nn.Linear(200,10)
    )

  def forward(self,x):
    return self.model(x)

#INSTANCE OF NN, LOSS, OPTIMIZER
clf = ImageClassifier().to('cuda') #extending to GPU
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn= nn.CrossEntropyLoss()

#Training Flow
if __name__ == "__main__":
  for epoch in range(10):
    train_accuracy=0
    for batch in train_loader:
      X,y= batch
      X,y=X.to('cuda'), y.to('cuda')
      yhat=clf(X) #prediction
      loss= loss_fn(yhat,y)
      yhat = yhat.data.max(dim=1, keepdim=True)[1]
      train_accuracy += yhat.eq(y.data.view_as(yhat)).cpu().sum()

      #apply backpropagation
      opt.zero_grad()
      loss.backward()
      opt.step() #gradient descent

    train_accuracy = train_accuracy/len(train_loader.dataset)*100
    print(f"Epoch:{epoch} loss is {loss.item()}, train accuracy is {train_accuracy}")
    with open ('model_state.pt','wb') as f:
      save(clf.state_dict(),f)

Epoch:0 loss is 0.07592614740133286, train accuracy is 96.24500274658203
Epoch:1 loss is 0.004146260675042868, train accuracy is 98.6816635131836
Epoch:2 loss is 0.0004097092314623296, train accuracy is 99.23999786376953
Epoch:3 loss is 0.0009940046584233642, train accuracy is 99.46833038330078
Epoch:4 loss is 8.233479456976056e-05, train accuracy is 99.56500244140625
Epoch:5 loss is 2.057309575320687e-05, train accuracy is 99.70166778564453
Epoch:6 loss is 1.4188201930664945e-05, train accuracy is 99.74666595458984
Epoch:7 loss is 3.307941005914472e-05, train accuracy is 99.76333618164062
Epoch:8 loss is 7.403718336718157e-05, train accuracy is 99.81500244140625
Epoch:9 loss is 8.813471140456386e-06, train accuracy is 99.82499694824219
